In [ ]:
import os
import json
import openai
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

base_url = os.environ.get("OPENAI_BASE_URL") or "http://localhost:11434/v1"
api_key = os.environ.get("OPENAI_API_KEY") or "ollama"
model_name = os.environ.get("MODEL_NAME") or "llama3.2" if api_key == "ollama" else "gpt-3.5-turbo"

client = openai.OpenAI(
    base_url = base_url,
    api_key=api_key,
)

with open('../booking.json', 'r') as f:
    booking = json.load(f)

def chat_with_gpt(
    messages,
    model=model_name,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
    temperature=0
):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            top_p=top_p,
            frequency_penalty=frequency_penalty,
            presence_penalty=presence_penalty,
            temperature=temperature
        )
        return response.choices[0].message.content
        
    except Exception as e:
        return str(e)

# Create chat input and button widgets
inp = widgets.Text(value="Hi", placeholder='Enter text here…')
chat_button = widgets.Button(description="Chat!")

# Output widget to display the interactive conversation
output = widgets.Output()
system_content = f"You are Shelley an automated travel assistant to answer questions about a customers holiday, \
start by greeting them by their first name and asking them how you can help with their holiday \
mention where they are going \
This customers holiday details are here: \
{booking}"

context = [{'role':'system', 'content':system_content}]

panels = []

def collect_messages():
    # Get the user input from the text input widget
    prompt = inp.value
    
    # Clear the text input widget
    inp.value = ''
    
    # Update the context with the user's message
    context.append({'role':'user', 'content': f"{prompt}"})
    
    # Get the assistant's response 
    response = chat_with_gpt(context)
    
    # Update the context with the assistant's response
    context.append({'role':'assistant', 'content': f"{response}"})
    
    # Create Markdown widgets for the user's message and assistant's response
    user_message = widgets.Output()
    with user_message:
        display(Markdown(f'**User:** {prompt}'))
    
    assistant_message = widgets.Output()
    with assistant_message:
        display(Markdown(f'**Assistant:** {response}'))
    
    # Append the new messages to the panels list
    panels.extend([user_message, assistant_message])
    
    # Clear the previous output and display the updated conversation
    with output:
        clear_output()
        for panel in panels:
            display(panel)

# Bind the button click event to the collect_messages function
chat_button.on_click(collect_messages)

# Create the dashboard layout
dashboard = widgets.VBox([inp, chat_button, output])

# Display the dashboard
display(dashboard)